In [ ]:
# Text text processing library
import torchtext
from torchtext.vocab import Vectors
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import itertools as it
from models import *
from helpers import *
debug = False
import time

In [ ]:
# Our input $x$
TEXT = torchtext.data.Field()

# Data distributed with the assignment
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path=".", 
    train="train.txt", validation="valid.txt", test="valid.txt", text_field=TEXT)

TEXT.build_vocab(train)
if debug:
    TEXT.build_vocab(train, max_size=1000)

train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=10, device=-1, bptt_len=32, repeat=False)

# Build the vocabulary with word embeddings
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
TEXT.vocab.load_vectors(vectors=Vectors('wiki.simple.vec', url=url))

In [111]:
it = iter(test_iter)
batch = next(it)
print(batch.text.size())
# print(batch.text[:,3])
print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:,4].data]))
print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:,5].data]))

torch.Size([32, 10])
makes some executives nervous <eos> last year the research and development division of weyerhaeuser co. the large <unk> concern invited a <unk> to its <unk> wash. offices <eos> phil <unk> a software
more expensive than direct treasury borrowing said rep. <unk> stark d. calif. the bill 's chief sponsor <eos> the complex financing plan in the s&l bailout law includes raising $ N billion


Perplexity goals:
count: 120-200
feedforward: 100-150
recurrent: below 100 (between 80-100)

In [245]:
train_iter, _, _  = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=1, device=-1, bptt_len=10000, repeat=False)
tgram = Trigram(TEXT)
tgram.train_counts(train_iter)
tgram.set_alpha(0.25, 0.25)

Iteration 0


In [269]:
_, val_iter, _  = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=1, device=-1, bptt_len=10000, repeat=False)
for (a1, a2) in [(0.3, 0.5)]: #it.product(np.arange(0.1, 1, 0.1), repeat=2):
    if a1 + a2 >= 1:
        continue
    tgram.set_alpha(a1, a2)
    le = LangEvaluator(tgram, TEXT, evalmetric='perplexity')
    print(a1, a2, le.evaluate(val_iter))

Iteration 0
0.3 0.5 191.339042201


In [332]:
params_train = list(filter(lambda p : p.requires_grad, model_nnlm.parameters()))
print([p.size() for p in params_train])

[torch.Size([10001, 300]), torch.Size([60, 1, 6, 300]), torch.Size([60]), torch.Size([10001, 1, 6, 300]), torch.Size([10001]), torch.Size([10001, 60]), torch.Size([10001])]


In [3]:
train_iter, val_iter, _  = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=10, device=-1, bptt_len=32, repeat=False)
model_nnlm = NNLM(TEXT)
trainer = LangTrainer(TEXT, model_nnlm)
le = LangEvaluator(model_nnlm, TEXT)
trainer.train(train_iter, num_iter=100, skip_iter=20, le=le, val_iter=val_iter)

Iteration 0, loss: 9.225080, norm: 0.000000, elapsed: 0.583086


TypeError: unsupported operand type(s) for -: 'str' and 'float'

In [4]:
_, val_iter, _  = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=10, device=-1, bptt_len=32, repeat=False)
le = LangEvaluator(model_nnlm, TEXT)
model_nnlm.eval()
le.evaluate(val_iter)

Iteration 0
Iteration 100
Iteration 200


9122.8357501977152

In [11]:
le = LangEvaluator2(model_nnlm, TEXT)
model_nnlm.eval()
le.evaluate(val_iter, num_iter=5)

Iteration 0
320 2903.7509765625
640 5812.7021484375
960 8721.039306640625
1280 11624.723388671875
1600 14529.80322265625
1920 17433.552978515625
2240 20345.4892578125


8802.6465923460746

In [12]:
b = time.time()
print(time.time() - a)

11.59984803199768


## IGNORE STUFF BELOW HERE

In [128]:
    # NOT USED!
    # Here arr is a 1-D numpy array; this returns 
    # groups of n consecutive words (with overlapping)
    def get_ngrams(self, arr, n=3):
        len_ngrams = arr.shape[0] - n + 1
        ngram_inds = np.tile(np.reshape(np.arange(len_ngrams), [len_ngrams, 1]), [1, n]) + \
                    np.tile(np.reshape(np.arange(n), [1, n]), [len_ngrams, 1])
        return np.take(arr, ngram_inds)

In [288]:
a = [1,2,3]
b = [2 * x + i for i,x in enumerate(a)]
print(b)

[2, 5, 8]


In [285]:
print(TEXT.vocab.vectors.size())

torch.Size([10001, 300])


In [7]:
print('Time diff: %f' % (time.time() - time.time()))

Time diff: 0.000000
